In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from datascience import Table
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Import files into dataframe
file_index = [1, 2, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

revisions_df = pd.DataFrame()

for i in tqdm(range(len(file_index))):
    revisions = pd.read_csv('revisions' + str(file_index[i]) + '_complete.csv')
    revisions_temp = pd.DataFrame(revisions)
    revisions_df = pd.concat([revisions_df, revisions_temp], axis=0)

In [ ]:
# Convert timestamp into readable format
import copy
from tqdm import tqdm
import datetime

new_df_combined = copy.deepcopy(revisions_df)

for index, row in tqdm(new_df_combined.iterrows()):
    try:
        time_string = row['timestamp']

        year = int(time_string[0:4])
        month = int(time_string[5:7])
        day = int(time_string[8:10])
        hour = int(time_string[11:13])
        minute = int(time_string[14:16])
        second = int(time_string[17:19])

        time_val = pd.to_datetime(datetime.datetime(year, month, day, hour, minute, second))
        time_placeholder = (datetime.datetime.utcfromtimestamp(0) - time_val).total_seconds()
        
        new_df_combined.loc[index, 'timestamp'] = time_placeholder
    except:
        new_df_combined.loc[index, 'timestamp'] = 0

new_df_combined.timestamp = new_df_combined.timestamp.astype(float)

# Combine pages to the same page by the same author; taking the last edited version of the page by the author
new_df_combined = new_df_combined.drop(['redirect', 'ns', 'rev_id', 'ip', 'username', 'comment', 'byt', 'rev_id_prev'], axis = 1) 
new_df_combined = new_df_combined.groupby(['page_id','page_title', 'country', 'org']).apply(lambda x: x.loc[x.timestamp.idxmax()]).reset_index(drop = True)

In [ ]:
# Drop unnecessary variables
revisions_df2 = new_df_combined.drop(['page_id', 'page_title', 'timestamp', 'old_text', 'org'], axis = 1)

# Convert text to string 
revisions_df2['new_text'] = revisions_df2['new_text'].astype(str)

# Aggregate data by country
revisions_df3 = revisions_df2.groupby(['country']).agg({'new_text':'sum'}).reset_index()

# Convert to datascience table
revisions_df3 = Table.from_df(revisions_df2)
revisions_df3.num_rows

In [ ]:
# Define function to count the number of mentions
import re

def mentions_country(word, show_table=False):
    count = []
    for i in np.arange(revisions_df3.num_rows):
        matches = re.findall('%s' % word, revisions_df3.column('new_text')[i])
        count.append(len(matches))
        
    mentions = revisions_df3.drop('new_text').with_columns('Count', count)
    if (show_table):
        mentions.show(8)
    return mentions

In [ ]:
# Define words to find and count
words = ['Australia', \
         'Austria', \
         'Brazil', \
         'Canada', 'Canadian', \
         'Chile', \
         'European Union','EU', \
         'Finland', \
         'German', \
         'Ireland', 'Irish' \
         'Israel', \
         'Italy', 'Italian' \
         'NATO', 'North Atlantic Treaty Organization'\
         'Netherlands', 'Dutch', 'Holland'\
         'New Zealand', \
         'Norway', 'Norwegian'\
         'Poland', 'Polish' \
         'Russia', 'Soviet'\
         'Sweden', 'Swedish'\
         'Switzerland', 'Swiss'\
         'Turkey', 'Turkish' \
         'UK', 'United Kingddom', 'England' \
         'Ukraine', 'Ukrainaian'\
         'UN', 'United Nations', \
         'USA', 'United States', 'US', 'North America', \
         'China', \
         'India', \
         'Japan',  \
         'France']

# Find words and append to table
table2 = Table().with_column('Country', revisions_df3[0])

for word in words:
    col = mentions_country(word, show_table=False)
    table2 = table2.with_columns(word, col[1])

In [ ]:
# Merge words for each country
table3 = table3.with_columns('Canada_merge', table3['Canada'] + table3['Canadian'])
table3 = table3.with_columns('EU_merge', table3['European Union'] + table3['EU'])
table3 = table3.with_columns('Ireland_merge', table3['Irish'] + table3['Ireland'])
table3 = table3.with_columns('Italy_merge', table3['Italy'] + table3['Italian'])
table3 = table3.with_columns('NATO_merge', table3['North Atlantic Treaty Organization'] + table3['NATO'])
table3 = table3.with_columns('Netherlands_merge', table3['Netherlands'] + table3['Dutch'] + table3['Holland'])
table3 = table3.with_columns('Norway_merge', table3['Norway'] + table3['Norwegian'])
table3 = table3.with_columns('Poland_merge', table3['Poland'] + table3['Polish'])
table3 = table3.with_columns('Russia_merge', table3['Russia'])# + table3['Soviet'])
table3 = table3.with_columns('Sweden_merge', table3['Sweden'] + table3['Swedish'])
table3 = table3.with_columns('Switzerland_merge', table3['Swiss'] + table3['Switzerland'])
table3 = table3.with_columns('Turkey_merge', table3['Turkey'] + table3['Turkish'])
table3 = table3.with_columns('UK_merge', table3['UK'] + table3['United Kingdom'] + table3['England'])
table3 = table3.with_columns('Ukraine_merge', table3['Ukraine'] + table3['Ukrainian'])
table3 = table3.with_columns('UN_merge', table3['UN'] + table3['United Nations'])
table3 = table3.with_columns('USA_merge', table3['USA'] + table3['United States'] + table3['US'] + table3['North America'])
table3 = table3.with_columns('China_merge', table3['China'] + table3['Chinese'])
table3 = table3.with_columns('France_merge', table3['France'] + table3['French'])
table3 = table3.drop(['Canada', 'Canadian', \
                      'European Union', 'EU', \
                      'Ireland', 'Irish', \
                      'Italy', 'Italian', \
                      'NATO', 'North Atlantic Treaty Organization', \
                      'Netherlands', 'Dutch', 'Holland', \
                      'Norway', 'Norwegian', \
                      'Poland', 'Polish', \
                      'Russia', 'Soviet', \
                      'Sweden', 'Swedish', \
                      'Switzerland', 'Swiss', \
                      'Turkey', 'Turkish',\
                      'UK', 'United Kingdom', 'England',\
                      'Ukraine', 'Ukrainian', \
                      'UN', 'United Nations', \
                      'USA', 'United States', 'US', 'North America', \
                      'China', 'Chinese', \
                      'France', 'French'])

table3 = table3.relabeled('Canada_merge', 'Canada')
table3 = table3.relabeled('EU_merge', 'EU')
table3 = table3.relabeled('Ireland_merge', 'Ireland')
table3 = table3.relabeled('Italy_merge', 'Italy')
table3 = table3.relabeled('NATO_merge', 'NATO')
table3 = table3.relabeled('Netherlands_merge', 'Netherlands')
table3 = table3.relabeled('Norway_merge', 'Norway')
table3 = table3.relabeled('Poland_merge', 'Poland')
table3 = table3.relabeled('Sweden_merge', 'Sweden')
table3 = table3.relabeled('Switzerland_merge', 'Switzerland')
table3 = table3.relabeled('Turkey_merge', 'Turkey')
table3 = table3.relabeled('UK_merge', 'UK')
table3 = table3.relabeled('Ukraine_merge', 'Ukraine')
table3 = table3.relabeled('UN_merge', 'UN')
table3 = table3.relabeled('USA_merge', 'USA')
table3 = table3.relabeled('China_merge', 'China')
table3 = table3.relabeled('France_merge', 'France')
table3 = table3.relabeled('Russia_merge', 'Russia')
table3 = table3.relabeled('German', 'Germany')
table3

In [ ]:
table3.to_csv('step_5_mentions_table3.csv')